In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# EXERCISE 1 SOLUTION
# Define a kernel on n qubits that
# maps |x> to |x+1> mod 2**n for arbitrary n

@cudaq.kernel
def inc_with_n_qubits(qubits: cudaq.qview):
    n = len(qubits)
    for i in range(n):
        if i + 1 < n:
            x.ctrl(qubits[i+1:n], qubits[i])
        else:
            x(qubits[i])

# Define a kernel on n qubits for the dec operation that
# maps |x> to |x-1> mod 2**n
@cudaq.kernel
def dec(qubits : cudaq.qview):
    cudaq.adjoint(inc_with_n_qubits, qubits)    

# A kernel to test your solution
@cudaq.kernel()
def test(binary_list : list[int]):
    qubits = cudaq.qvector(len(binary_list))
    # Initialize the qubits to the initial position
    for n in range(len(binary_list)):
        if binary_list[n] == 1:
            x(qubits[n])
    # Move from position |x> to |x+1>
    inc_with_n_qubits(qubits)

# Code to verify that the kernel acts as required
def int_to_binary_list(n, num_bits):
    """
    Convert a non-negative integer to a fixed-length list of binary digits.

    Parameters:
    n (int): A non-negative integer to be converted.
    num_bits (int): The desired fixed length of the binary list.

    Returns:
    List[int]: A list containing the binary digits of the integer, 
               padded with leading zeros to match num_bits.

    Raises:
    ValueError: If n is negative or cannot be represented in num_bits bits.
    """
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")
    if num_bits <= 0:
        raise ValueError("Number of bits must be a positive integer.")
    
    # Convert integer to binary string without '0b' prefix
    binary_str = bin(n)[2:]
    
    # Check if the binary representation fits in num_bits
    if len(binary_str) > num_bits:
        raise ValueError(f"{n} cannot be represented in {num_bits} bits.")
    
    # Pad the binary string with leading zeros to match num_bits
    binary_str_padded = binary_str.zfill(num_bits)
    
    # Convert the binary string to a list of integers
    binary_list = [int(bit) for bit in binary_str_padded]
    
    return binary_list

print(int_to_binary_list(2,5))

num_qubits = 5
for i in range(2**num_qubits):
    binary_input = int_to_binary_list(i, num_qubits)
    result = cudaq.sample(test, binary_input).most_probable()
    print(f"Initial position: {i:02d}, Result: {result}")
    

In [ ]:
# EXERCISE 2 SOLUTION
# Set the number of qubits Do Not Change without changing inc and dec
num_qubits = 4

# Set the number of steps
num_time_steps = 1 #CHANGE_ME

# Kernel for one step of the SSQW without measurement
@cudaq.kernel
def SSQW_one_step(walker_qubits : cudaq.qview, coin_qubit : cudaq.qubit, endpoint_qubit : cudaq.qubit):
        
    # One quantum walk step
    
    # First flip of the coin with the first coin operation, F1
    x(coin_qubit)      #CHANGE_ME
    
    # Walker's position change
    # Avoid shifting from |1111> to |0000> in case the coin is |1> by flipping the coin to |0>
    no_INC_at_right_endpoint(walker_qubits, coin_qubit, endpoint_qubit)
    
    # Shift to the right if the coin is |1> and the position is not |1111>
    cudaq.control(inc, coin_qubit, walker_qubits)
    
    # Reset the coin and endpoints in case they were changed to avoid moving from |1111> to |0000>
    reset_coin_and_endpoint(coin_qubit, endpoint_qubit)
    
    
    # Second flip of the coin with the second coin operation, F2
    h(coin_qubit)      #CHANGE_ME
    
    # Controlled-decrementer
    # Avoid shifting from |0000> to |1111> in case the coin is |0> by flipping the coin to |1>
    no_DEC_at_left_endpoint(walker_qubits, coin_qubit, endpoint_qubit)
    
    # Apply the dec if the the coin is in |0>
    x(coin_qubit)
    cudaq.control(dec, coin_qubit, walker_qubits)
    x(coin_qubit)
    
    # Reset the coin and endpoints in case they were changed to avoid moving from |0000> to |1111>
    reset_coin_and_endpoint(coin_qubit, endpoint_qubit)
    
@cudaq.kernel()
def SSQW_with_measurement(num_qubits : int, num_time_steps : int):
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qubit()
    endpoint_qubit = cudaq.qubit()
    
    # Initial walker state 1/sqrt(2)(|1000> + |0000>) #CHANGE_ME
    #h(walker_qubits[0])
    #x(walker_qubits)
    
        
    # Initial coin state    #CHANGE_ME
    x(coin_qubit)
    
    for _ in range(num_time_steps):
        SSQW_one_step(walker_qubits, coin_qubit, endpoint_qubit)
    
    # Measure the walker's state 
    mz(walker_qubits)

#cudaq.draw(SSQW_with_measurement, num_qubits, num_time_steps)

# Sample the kernel for the quantum walk
result = cudaq.sample(SSQW_with_measurement, num_qubits, num_time_steps, shots_count=5000)


title = 'Sampling SSQW with {} time steps'.format(num_time_steps)
plot_walk_results(result, num_qubits, title)